## Segmentations Data Analysis

The first analysis is to correctly relate the dicom files with the segmentation files, in order to know the matched patients that have available segmentations from their respective imaging studies.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import pydicom

In [ ]:
metadata_path = "/kaggle/input/rsna-2023-abdominal-trauma-detection/train_series_meta.csv"

train_metadata = pd.read_csv(metadata_path)
train_metadata.head()

In [ ]:
segmentations_path = "/kaggle/input/rsna-2023-abdominal-trauma-detection/segmentations"

segmentations = os.listdir(segmentations_path)
segmentations = [int(os.path.splitext(segmentation)[0]) for segmentation in segmentations]
len(segmentations)

In [ ]:
series = train_metadata["series_id"].tolist()

In [ ]:
matched_series = []

for segmentation in segmentations:
    if segmentation in series:
        matched_series.append(segmentation)
    else:
        pass 

In [ ]:
len(matched_series)

In [ ]:
patients_segment = train_metadata[train_metadata["series_id"].isin(matched_series)].reset_index(drop=True)
patients_segment

In [ ]:
patients_with_segmentations = patients_segment["patient_id"].unique()
patients_with_segmentations

## Reading the DICOM and NIFTII files

In [ ]:
import numpy as np 
import pandas as pd 
import pydicom 
import nibabel as nib
import matplotlib.pyplot as plt
import seaborn as sn
import os
from glob import glob

In [ ]:
PATH = '/kaggle/input/rsna-2023-abdominal-trauma-detection'

train = pd.read_csv(f'{PATH}/train.csv')
train_series_meta = pd.read_csv(f'{PATH}/train_series_meta.csv')
df = pd.merge(train_series_meta, train, how='inner', on='patient_id')
df

In [ ]:

def visualize_dicom_images(patient_id, series_id, idx=0):
    patient_id = patient_id[idx]
    series_id = series_id[idx]
    
    image_file = glob(f"/kaggle/input/rsna-2023-abdominal-trauma-detection/train_images/{patient_id}/{series_id}/*.dcm")
    plt.figure(figsize=(20, 20))

    for i in range(8):
        ax = plt.subplot(4, 4, i + 1)
        image_path = image_file[i]
        ds = pydicom.dcmread(image_path)
    
        plt.axis('off')
        plt.imshow(ds.pixel_array,cmap='gray')

    
visualize_dicom_images(df["patient_id"], df["series_id"], idx=24)

In [ ]:
seg_image_file = glob("/kaggle/input/rsna-2023-abdominal-trauma-detection/segmentations/*.nii")

plt.figure(figsize=(20, 20))

for i in range(8):
    ax = plt.subplot(4, 4, i + 1)
    image_path = seg_image_file[i]
    nii_img = nib.load(image_path).get_fdata()
    nib_image = nii_img[:,:,nii_img.shape[2]//2]
    
    plt.axis('off')
    plt.imshow(nib_image,cmap='gray')

In [ ]:
def create_3D_scans(folder, downsample_rate=1):
    
    #reads the filenames in the folder, extracts the IDs from the filenames, sorts them, and constructs a list of filenames to read in order.
    filenames = os.listdir(folder)
    filenames = [int(filename.split('.')[0]) for filename in filenames]
    filenames = sorted(filenames)
    filenames = [str(filename) + '.dcm' for filename in filenames]
        
    volume = []
    for filename in filenames[::downsample_rate]:
        filepath = os.path.join(folder, filename)
        ds = pydicom.dcmread(filepath)
        #This extracts the pixel array data from the DICOM object. The pixel array represents the 2D image slice
        image = ds.pixel_array
        
        # find rescale params
        if ("RescaleIntercept" in ds) and ("RescaleSlope" in ds):
            intercept = float(ds.RescaleIntercept)
            slope = float(ds.RescaleSlope)
    
        # find clipping params
        center = int(ds.WindowCenter)
        width = int(ds.WindowWidth)
        low = center - width / 2
        high = center + width / 2    
        image = (image * slope) + intercept
        image = np.clip(image, low, high)

        image = (image / np.max(image) * 255).astype(np.int16)#Normalize the pixel values to a range between 0 and 255 and convert them to 16-bit integers. This step is usually done for visualization purpose
        image = image[::downsample_rate, ::downsample_rate]
        volume.append( image )
    
    volume = np.stack(volume, axis=0)
    return volume

In [ ]:
def create_3D_segmentations(filepath, downsample_rate=1):
    img = nib.load(filepath).get_fdata()
    img = np.transpose(img, [2, 1, 0])
    img = np.rot90(img, -1, (1,2))
    img = img[::-1,:,:]
    img = np.transpose(img, [2, 1, 0])
    img = img[::downsample_rate, ::downsample_rate, ::downsample_rate]
    return img

In [ ]:
def segmentation_visualization(patient_id, series_id, idx):
    
    PATH = '/kaggle/input/rsna-2023-abdominal-trauma-detection'
    seg_filepath = f'{PATH}/segmentations/{series_id[idx]}.nii'
    img_filepath = f'{PATH}/train_images/{patient_id[idx]}/{series_id[idx]}'
    
    a = glob(f'{PATH}/segmentations/*.nii')

    if f'{PATH}/segmentations/{series_id[idx]}.nii' in a:
        print('The series id exists')
    else:
        print('The series id does not exist')
    
    volume = create_3D_scans(img_filepath)
    volume = volume.transpose(1, 2, 0)
    print(volume.shape)
    
    volume_seg = create_3D_segmentations(seg_filepath)
    print(volume_seg.shape)
    
    fig = plt.figure(figsize=(16,16), constrained_layout=True)
    #fig.suptitle(f"Analyzing patient {patient_id[idx]} and respective series {series_id[idx]}")

    ax1 = fig.add_subplot(131)
    ax1.imshow(volume[:,:,0], cmap = 'gray')
    ax1.set_title(f'Original Image -> patient_id:{patient_id[idx]}, series_id:{series_id[idx]}', fontsize=14)

    ax2 = fig.add_subplot(132)
    ax2.imshow(volume_seg[:,:,0], cmap = 'gray')
    ax2.set_title('Segmented Image', fontsize=14)

    ax3 = fig.add_subplot(133)
    ax3.imshow(volume[:,:,0]*np.where(volume_seg[:,:,0]>0,1,0), cmap = 'gray')
    ax3.set_title('Overlay of Original and Segmented', fontsize=14)
    plt.show()

In [ ]:
segmentation_visualization(patients_segment["patient_id"], patients_segment["series_id"], idx=200)

## Animation

In [ ]:
from matplotlib import animation, rc
rc('animation', html='jshtml')

In [ ]:
def create_animation_dicom(patient_id, series_id, idx):
    
    PATH = '/kaggle/input/rsna-2023-abdominal-trauma-detection'
    img_filepath = f'{PATH}/train_images/{patient_id[idx]}/{series_id[idx]}'
    
    volume = create_3D_scans(img_filepath)
    volume = volume.transpose(1, 2, 0)
    
    num_frames = volume.shape[2]
    fig = plt.figure(figsize=(8, 8))
    im = plt.imshow(volume[:, :, 0])
    text = plt.text(0.05, 0.05, f'Slice {0}', transform=fig.transFigure, fontsize=14, color='blue')
    plt.title(f"Original Image -> patient_id:{patient_id[idx]}, series_id:{series_id[idx]}", fontsize=16)

    def animate_func(i):
        im.set_array(volume[:, :, i])
        text.set_text(f'Slice {i}')
        return [im]

    plt.axis('off')
    plt.gray()
    plt.close()

    return animation.FuncAnimation(fig, animate_func, frames=num_frames, interval=1000 // 10)


In [ ]:
def create_animation_niftii(patient_id, series_id, idx):
    
    PATH = '/kaggle/input/rsna-2023-abdominal-trauma-detection'
    seg_filepath = f'{PATH}/segmentations/{series_id[idx]}.nii'
    
    a = glob(f'{PATH}/segmentations/*.nii')

    if f'{PATH}/segmentations/{series_id[idx]}.nii' in a:
        print('The series id exists')
    else:
        print('The series id does not exist')
    
    volume_seg = create_3D_segmentations(seg_filepath)
    
    num_frames = volume_seg.shape[2]
    fig = plt.figure(figsize=(8, 8))
    im = plt.imshow(volume_seg[:, :, 0])
    text = plt.text(0.05, 0.05, f'Slice {0}', transform=fig.transFigure, fontsize=14, color='blue')
    plt.title(f'Segmentation -> patient_id:{patient_id[idx]}, series_id:{series_id[idx]}', fontsize=16)

    def animate_func(i):
        im.set_array(volume_seg[:, :, i])
        text.set_text(f'Slice {i}')
        return [im]

    plt.axis('off')
    plt.gray()
    plt.close()

    return animation.FuncAnimation(fig, animate_func, frames=num_frames, interval=1000 // 10)

In [ ]:
create_animation_dicom(patients_segment["patient_id"], patients_segment["series_id"], idx=200)

In [ ]:
create_animation_niftii(patients_segment["patient_id"], patients_segment["series_id"], idx=200)